# Setup

In [1]:
import sys
import importlib

sys.path.insert(0, "../PopV")
import popv
from popv.reproducibility import _accuracy

2023-03-11 23:23:13.765594: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-11 23:23:14.025757: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-11 23:23:15.117483: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-03-11 23:23:15.117548: W tensorflow/compiler/xla/stream_executor/platf

In [2]:
popv.__file__

'/data/extra_files/galen_dropbox_figures/can_reproducibility/lung_models/../PopV/popv/__init__.py'

In [3]:
import anndata
import scanpy as sc
import numpy as np
import obonet
import networkx as nx
import pandas as pd
import time

In [4]:
from popv._utils import make_ontology_dag

In [6]:
obo_fn = '../PopV/ontology/cl.obo'

G = popv._utils.make_ontology_dag(obo_fn)

# Lung


In [7]:
fn = 'lung_annotation/annotation/query_and_reference_popv.h5ad'
adata = anndata.read(fn)

In [8]:
adata

AnnData object with n_obs × n_vars = 105322 × 4000
    obs: 'tissue_in_publication', 'donor_id', 'free_annotation', 'compartment', 'cell_type_ontology_term_id', 'cell_type', 'assay', 'sex', 'tissue', 'cell_ontology_class', 'cell_ontology_class_tissue', 'donor_tissue', 'assay_correct', 'donor_assay', '_batch_annotation', '_labels_annotation', '_ref_subsample', 'paper_annotation', 'sample', '_dataset', 'n_counts', 'popv_celltypist_prediction', 'popv_celltypist_prediction_probabilities', 'popv_knn_on_bbknn_prediction', 'popv_knn_on_bbknn_prediction_probabilities', 'popv_knn_on_scanorama_prediction', 'popv_knn_on_scanorama_prediction_probabilities', 'subsampled_labels', '_scvi_batch', '_scvi_labels', 'popv_knn_on_scvi_prediction', 'popv_knn_on_scvi_prediction_probabilities', '_labels_annotation_cell_ontology_id', 'popv_onclass_prediction', 'popv_onclass_seen', 'popv_onclass_prediction_probabilities', 'popv_onclass_seen_probabilities', 'popv_rf_prediction', 'popv_rf_prediction_probabilities

In [14]:
df = pd.read_csv('lung_annotation/seurat_annotation/seurat_predictions.csv')
df['seurat_prediction'] = df["predicted.id"]
df['seurat_probability'] = df["prediction.score.max"]
df.index = df["Unnamed: 0"]
adata.obs['seurat_prediction'] = adata.obs['cell_ontology_class']
adata.obs['seurat_probability'] = 1
adata.obs.loc[adata.obs['_dataset']=='query', 'seurat_prediction'] = df.loc[adata[adata.obs['_dataset']=='query'].obs_names, 'seurat_prediction']
adata.obs.loc[adata.obs['_dataset']=='query', 'seurat_probability'] = df.loc[adata[adata.obs['_dataset']=='query'].obs_names, 'seurat_probability']

In [15]:
pred_keys = list(set(adata.uns['prediction_keys']).union(set(adata.uns['prediction_keys_seen']))) + ['seurat_prediction']

cts = np.unique(adata.obs[pred_keys])

bad_cts = []

for ct in cts:
    if ct not in G:
        bad_cts.append(ct)
    elif 'cell' not in nx.shortest_path_length(G, ct):
        bad_cts.append(ct)
        
cts = np.unique(adata.obs['cell_ontology_class'])

for ct in cts:
    if ct not in G:
        print('2', ct)
        bad_cts.append(ct)
    elif 'cell' not in nx.shortest_path_length(G, ct):
        print('1', ct)
        bad_cts.append(ct)

In [16]:
pred_keys

['popv_celltypist_prediction',
 'popv_onclass_prediction',
 'popv_scanvi_prediction',
 'popv_rf_prediction',
 'popv_knn_on_scanorama_prediction',
 'popv_svm_prediction',
 'popv_onclass_seen',
 'popv_knn_on_scvi_prediction',
 'popv_knn_on_bbknn_prediction',
 'seurat_prediction']

In [17]:
# should be empty

bad_cts

[]

## Fine Accuracy

### ontology_vote

In [18]:
importlib.reload(_accuracy)

<module 'popv.reproducibility._accuracy' from '/data/extra_files/galen_dropbox_figures/can_reproducibility/lung_models/../PopV/popv/reproducibility/_accuracy.py'>

In [19]:
paths = nx.algorithms.simple_paths.shortest_simple_paths(
                    nx.Graph(G), source='pulmonary ionocyte', target='macrophage'
                )

In [20]:
_accuracy._fine_ontology_sibling_accuracy(
    adata, 
    obofile=obo_fn,
    gt_key='cell_ontology_class', 
    pred_key='popv_prediction')

In [21]:
adata.obs

,tissue_in_publication,donor_id,free_annotation,compartment,cell_type_ontology_term_id,cell_type,assay,sex,tissue,cell_ontology_class,...,popv_majority_vote_prediction,popv_majority_vote_score,popv_prediction,popv_prediction_score,popv_prediction_depth,popv_prediction_onclass_relative_depth,popv_parent,seurat_prediction,seurat_probability,popv_prediction_ontology_distance
AAACCCAAGAACTCCT_TSP14_Lung_Distal_10X_1_1,Lung,TSP14,type ii pneumocyte,epithelial,CL:0002063,type II pneumocyte,10x 3' v3,male,lung,type II pneumocyte,...,type II pneumocyte,6,type II pneumocyte,5,5,3,pneumocyte,type II pneumocyte,1.000000,0
TCGGGCATCACGAACT_TSP14_Lung_Proximal_10X_1_1,Lung,TSP14,type ii pneumocyte,epithelial,CL:0002063,type II pneumocyte,10x 3' v3,male,lung,type II pneumocyte,...,type II pneumocyte,7,type II pneumocyte,6,5,3,pneumocyte,type II pneumocyte,1.000000,0
TCGGGCATCTCCCTAG_TSP14_Lung_Proximal_10X_1_1,Lung,TSP14,type ii pneumocyte,epithelial,CL:0002063,type II pneumocyte,10x 3' v3,male,lung,type II pneumocyte,...,type II pneumocyte,8,type II pneumocyte,7,5,3,pneumocyte,type II pneumocyte,1.000000,0
TCGGGTGAGGTTAGTA_TSP14_Lung_Proximal_10X_1_1,Lung,TSP14,type ii pneumocyte,epithelial,CL:0002063,type II pneumocyte,10x 3' v3,male,lung,type II pneumocyte,...,type II pneumocyte,8,type II pneumocyte,7,5,3,pneumocyte,type II pneumocyte,1.000000,0
TCGGGTGGTGTTTGCA_TSP14_Lung_Proximal_10X_1_1,Lung,TSP14,type ii pneumocyte,epithelial,CL:0002063,type II pneumocyte,10x 3' v3,male,lung,type II pneumocyte,...,type II pneumocyte,8,type II pneumocyte,7,5,3,pneumocyte,type II pneumocyte,1.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K23_B003269.gencode.vH29,NaN,3,NaN,immune,NaN,NaN,Smart-seq2,NaN,lung,mature NK T cell,...,mature NK T cell,6,mature NK T cell,5,10,-1,mature alpha-beta T cell,mature NK T cell,1.000000,0
K21_B003269.gencode.vH29,NaN,3,NaN,immune,NaN,NaN,Smart-seq2,NaN,lung,mature NK T cell,...,"CD8-positive, alpha-beta T cell",8,"CD8-positive, alpha-beta T cell",7,10,-1,"CD8-positive, alpha-beta T cell",mature NK T cell,0.586027,2_sib
B4_B002460.gencode.vH29,NaN,3,NaN,epithelial,NaN,NaN,Smart-seq2,NaN,lung,type II pneumocyte,...,type II pneumocyte,8,type II pneumocyte,7,5,3,pneumocyte,type II pneumocyte,1.000000,0
K19_B003269.gencode.vH29,NaN,3,NaN,immune,NaN,NaN,Smart-seq2,NaN,lung,mature NK T cell,...,mature NK T cell,7,mature NK T cell,6,10,-1,mature NK T cell,mature NK T cell,0.780940,0


### all other methods

In [22]:
pred_keys

['popv_celltypist_prediction',
 'popv_onclass_prediction',
 'popv_scanvi_prediction',
 'popv_rf_prediction',
 'popv_knn_on_scanorama_prediction',
 'popv_svm_prediction',
 'popv_onclass_seen',
 'popv_knn_on_scvi_prediction',
 'popv_knn_on_bbknn_prediction',
 'seurat_prediction']

In [23]:
for k in pred_keys:
    print(k)
    _accuracy._fine_ontology_sibling_accuracy(
        adata, 
        obofile=obo_fn,
        gt_key='cell_ontology_class', 
        pred_key=k)

popv_celltypist_prediction
popv_onclass_prediction
popv_scanvi_prediction
popv_rf_prediction
popv_knn_on_scanorama_prediction
popv_svm_prediction
popv_onclass_seen
popv_knn_on_scvi_prediction
popv_knn_on_bbknn_prediction
seurat_prediction


In [27]:
adata.write('lung_annotation/annotation/query_and_reference_popv_fine_ontology_accuracy.h5ad')

## Coarse Accuracy

### ontology_vote

In [28]:
adata.obs.drop(labels=[i for i in adata.obs.columns if 'ontology_distance' in i], axis=1, inplace=True)
adata

AnnData object with n_obs × n_vars = 105322 × 4000
    obs: 'tissue_in_publication', 'donor_id', 'free_annotation', 'compartment', 'cell_type_ontology_term_id', 'cell_type', 'assay', 'sex', 'tissue', 'cell_ontology_class', 'cell_ontology_class_tissue', 'donor_tissue', 'assay_correct', 'donor_assay', '_batch_annotation', '_labels_annotation', '_ref_subsample', 'paper_annotation', 'sample', '_dataset', 'n_counts', 'popv_celltypist_prediction', 'popv_celltypist_prediction_probabilities', 'popv_knn_on_bbknn_prediction', 'popv_knn_on_bbknn_prediction_probabilities', 'popv_knn_on_scanorama_prediction', 'popv_knn_on_scanorama_prediction_probabilities', 'subsampled_labels', '_scvi_batch', '_scvi_labels', 'popv_knn_on_scvi_prediction', 'popv_knn_on_scvi_prediction_probabilities', '_labels_annotation_cell_ontology_id', 'popv_onclass_prediction', 'popv_onclass_seen', 'popv_onclass_prediction_probabilities', 'popv_onclass_seen_probabilities', 'popv_rf_prediction', 'popv_rf_prediction_probabilities

In [29]:
importlib.reload(_accuracy)

<module 'popv.reproducibility._accuracy' from '/data/extra_files/galen_dropbox_figures/can_reproducibility/lung_models/../PopV/popv/reproducibility/_accuracy.py'>

In [30]:
_accuracy._ontology_accuracy(
    adata, 
    obofile=obo_fn,
    gt_key='cell_ontology_class', 
    pred_key='popv_prediction',
    save_key=f'consensus_ontology_accuracy')

In [34]:
2

2

In [33]:
adata.obs['consensus_ontology_accuracy'].value_counts()

exact       95016
no match     4991
sibling      3135
parent       2178
child           2
Name: consensus_ontology_accuracy, dtype: int64

In [35]:
pd.set_option('display.max_rows', 500)

In [36]:
df = adata.obs[adata.obs['consensus_ontology_accuracy']=='no match'].copy()
df['Count'] = 1
df = df.groupby(['popv_prediction', 'cell_ontology_class']).Count.count().reset_index()
df[df['Count']>10]

,popv_prediction,cell_ontology_class,Count
55,"CD4-positive, alpha-beta T cell","effector CD8-positive, alpha-beta T cell",65
95,"CD8-positive, alpha-beta T cell","effector CD4-positive, alpha-beta T cell",949
173,alveolar capillary type 2 endothelial cell,capillary endothelial cell,783
184,alveolar capillary type 2 endothelial cell,lung microvascular endothelial cell,11
209,basal cell,alveolar capillary type 2 endothelial cell,12
214,basal cell,capillary endothelial cell,18
216,basal cell,club cell,21
239,basal cell,respiratory goblet cell,26
268,basophil,macrophage,53
307,blood vessel endothelial cell,lung microvascular endothelial cell,16


### all other methods

In [37]:
pred_keys

['popv_celltypist_prediction',
 'popv_onclass_prediction',
 'popv_scanvi_prediction',
 'popv_rf_prediction',
 'popv_knn_on_scanorama_prediction',
 'popv_svm_prediction',
 'popv_onclass_seen',
 'popv_knn_on_scvi_prediction',
 'popv_knn_on_bbknn_prediction',
 'seurat_prediction']

In [38]:
for k in pred_keys:
    print(k)
    _accuracy._ontology_accuracy(
        adata, 
        obofile=obo_fn,
        gt_key='cell_ontology_class', 
        pred_key=k,
        save_key=f'{k}_ontology_accuracy')

popv_celltypist_prediction
popv_onclass_prediction
popv_scanvi_prediction
popv_rf_prediction
popv_knn_on_scanorama_prediction
popv_svm_prediction
popv_onclass_seen
popv_knn_on_scvi_prediction
popv_knn_on_bbknn_prediction
seurat_prediction


In [39]:
adata.write('lung_annotation/annotation/query_and_reference_final.h5ad')

... storing 'popv_celltypist_prediction_ontology_accuracy' as categorical
... storing 'popv_onclass_prediction_ontology_accuracy' as categorical
... storing 'popv_scanvi_prediction_ontology_accuracy' as categorical
... storing 'popv_rf_prediction_ontology_accuracy' as categorical
... storing 'popv_knn_on_scanorama_prediction_ontology_accuracy' as categorical
... storing 'popv_svm_prediction_ontology_accuracy' as categorical
... storing 'popv_onclass_seen_ontology_accuracy' as categorical
... storing 'popv_knn_on_scvi_prediction_ontology_accuracy' as categorical
... storing 'popv_knn_on_bbknn_prediction_ontology_accuracy' as categorical
... storing 'seurat_prediction_ontology_accuracy' as categorical
